In [ ]:
from google.colab import drive
# mount drive to access data
drive.mount('/content/drive')
# load data folder into working directory
!cp -r drive/MyDrive/data .
!cp -r drive/MyDrive/preprocessing.py .

### Imports

In [ ]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
import pandas as pd
import matplotlib.pyplot as plt
import string
import re
import preprocessing as p
from nltk.util import bigrams
from itertools import compress
import numpy as np
import math

In [ ]:
# loading raw data
raw_data = pd.read_csv('data/TweetsCOV19.csv')
# loading data with preprocessed tweets
data = pd.read_csv('data/cleaned_tweets.csv')

In [3]:
# tokenize the preprocessed tweets
tweets = data['TweetText'].tolist()
tokenized_tweets = p.tokenization(tweets)

KeyboardInterrupt: 

In [ ]:
# returns ngrams, ordered by frequency
def ngram_freq(tweets, n):

    ngrams = dict()

    for tweet in tweets:
        for i in range(len(tweet)-n + 1):
            if tuple(tweet[i:i+n]) in ngrams:
                ngrams[tuple(tweet[i:i+n])] += 1
            else:
                ngrams[tuple(tweet[i:i+n])] = 1

    ngrams = sorted(ngrams.items(), key=lambda x:-x[1])

    return ngrams

In [ ]:
# remove nans from raw_data
raw_data = raw_data[raw_data['TweetText'].notnull()]

In [ ]:
# turn tweets into words
raw_tweets = raw_data['TweetText'].tolist()
for i, tweet in enumerate(raw_tweets):
  raw_tweets[i] = tweet.split()

In [ ]:
# get most frequent unigrams and bigrams
unigrams = ngram_freq(tokenized_tweets, 1)
raw_unigrams = ngram_freq(raw_tweets, 1)
bigrams = ngram_freq(tokenized_tweets, 2)
raw_bigrams = ngram_freq(raw_tweets, 2)

In [ ]:
# plot unigram, bigram frequency before and after preprocessing 
fig, ax = plt.subplots(2, 2, sharey='row', figsize=(10,8))

titles = ['Most common unigrams before preprocessing', \
          'Most common unigrams after preprocessing', \
          'Most common bigrams before preprocessing', \
          'Most common bigrams after preprocessing']
grams = [raw_unigrams, unigrams, raw_bigrams, bigrams]
pos = [i for i in range(10)] # position of bars

for i, (title, gram) in enumerate(zip(titles, grams)):

    words = [gram[j][0][0] for j in range(10)]
    if i > 1:
        words = [' '.join(gram[j][0]) for j in range(10)]
    freq = [gram[j][1] for j in range(10)]

    ax[i//2][i % 2].bar(pos, freq, align='center', color='b', alpha=0.6)
    ax[i//2][i % 2].set_xticks(pos)
    ax[i//2][i % 2].set_xticklabels(words, fontsize=12, rotation='vertical')
    ax[i//2][i % 2].set_title(title, fontsize=12)

plt.tight_layout()
plt.savefig("uni-bigram-frequencies.png")
plt.show()

In [ ]:
# get sentiment frequencies
def sentiment_freq(data):

  sentiments = data['Sentiment'].tolist()
  freqs = dict()
  for s in sentiments:
    p, n = s.split()
    p = int(p)
    n = int(n)
    if p in freqs:
      freqs[p] += 1
    else:
      freqs[p] = 1
    if n in freqs:
      freqs[n] += 1
    else:
      freqs[n] = 1

  # turn dictionary into list
  sentlist = list(freqs.items())

  # order by sentiment for plot
  sentlist.sort(key=lambda x: x[0])

  return sentlist

In [ ]:
sentlist = sentiment_freq(data)
sentiment = [str(x[0]) for x in sentlist]
sent_freq = pd.Series([x[1] for x in sentlist])

plt.figure(figsize=(7, 5))
ax = sent_freq.plot(kind="bar", width=0.7, color='b', alpha=0.6)
ax.set_title("Sentiment Frequencies")
ax.set_xlabel("Sentiment")
ax.set_ylabel("Frequency")
ax.set_xticklabels(sentiment)

rects = ax.patches
labels = [str(x[1]) for x in sentlist]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(
        rect.get_x() + rect.get_width() / 2, height + 5, label, ha="center", \
         va="bottom"
    )
plt.savefig("sentiment-distribution.png")
plt.show()

In [ ]:
def sentiment_word_count_distribution(data):

  sentiments = data['Sentiment'].tolist()
  # split into positive and negative sentiments
  positive = []
  negative = []

  for s in sentiments:
    p, n = s.split()
    p = int(p)
    n = int(n)
    positive.append(p)
    negative.append(n)

  # select very positive, very negative and neutral sentiments
  pos_five = [x == 5 or x == 4 for x in positive]
  neg_five = [x == -5 or x == -4 for x in negative]
  neutral = [x == 1 and y == -1 for x,y in zip(positive, negative)]

  positive_tweets = list(compress(tokenized_tweets, pos_five))
  negative_tweets = list(compress(tokenized_tweets, neg_five))
  neutral_tweets = list(compress(tokenized_tweets, neutral))

  count_positive = [len(x) for x in positive_tweets]
  count_negative = [len(x) for x in negative_tweets]
  count_neutral = [len(x) for x in neutral_tweets]

  # normalize in range 1 to 0

  return [count_negative, count_neutral, count_positive]

In [ ]:
counts = sentiment_word_count_distribution(data)

In [ ]:
def plot_wordcount_histogram(ax, counts, title):

  d = np.diff(np.unique(counts)).min()
  left_of_first_bin = min(counts) - float(d)/2
  right_of_last_bin = max(counts) + float(d)/2
  ax.hist(counts, np.arange(left_of_first_bin, right_of_last_bin + d, d),\
          density=True, color='b', alpha=0.6)
  ax.set_title(title, fontsize=10)
  ax.set_xlabel("Word count")

In [ ]:
fig, ax = plt.subplots(1, 3, sharey='row', figsize=(12,4))
titles = ["Negative (-4, -5)", "Neutral (-1, +1)", "Positive (+4, +5)"]
for axis, count, title in zip(ax, counts, titles):
  plot_wordcount_histogram(axis, count, title)
fig.suptitle("Word count density as a function of sentiment")
plt.tight_layout()
plt.savefig("wordcount-densities.png")